In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI # The official OpenAI Python client
import os

# 1. Initialize the client (Requires setting OPENAI_API_KEY environment variable)
client = OpenAI(api_key="")

# --- Your Data ---
ai_summary = "A new chip design using quantum tunneling transistors could boost CPU efficiency by 40%."
articles = [
    "The latest article on CPU architecture details how Intel's new 10nm process improves power efficiency.",
    "Quantum computing breakthroughs lead to the first stable 50-qubit system, primarily focused on research.", 
    # ... all other articles
]

# 2. Prepare the texts for the API call
# OpenAI models do NOT require the 'query:' or 'passage:' prefixes.
texts_to_embed = [ai_summary] + articles

# 3. Call the OpenAI Embeddings API
try:
    response = client.embeddings.create(
        input=texts_to_embed,
        model="text-embedding-3-small" # Excellent performance and cost-effective
    )
    
    # 4. Extract the vectors and convert to NumPy array
    # The API returns a list of embedding objects, extract the vector list from each
    all_embeddings = np.array([
        item.embedding for item in response.data
    ])

except Exception as e:
    print(f"OpenAI API Error: {e}")
    # Handle API errors gracefully in your web backend
    
# 5. Separate and calculate (Identical to your original code!)
summary_embedding = all_embeddings[0].reshape(1, -1) 
article_embeddings = all_embeddings[1:]

# Calculate Cosine Similarity
relevance_scores = cosine_similarity(summary_embedding, article_embeddings)[0]

# ... (Rest of your scoring/ranking logic remains the same) ...


# --- 3. Display Results ---
results = []
for score, article in zip(relevance_scores, articles):
    # Format the score to two decimal places
    results.append({
        "article": article,
        "relevance_score": round(score, 4) 
    })

# Sort the results by score
ranked_results = sorted(results, key=lambda x: x['relevance_score'], reverse=True)

## Results Table
print("\n--- Relevance Scores (Summary vs. Articles) ---")
print("| Rank | Relevance Score | Article Snippet |")
print("|------|-------------------|-----------------|")
for i, res in enumerate(ranked_results):
    snippet = res['article'][:60] + "..." if len(res['article']) > 60 else res['article']
    print(f"| {i+1}    | **{res['relevance_score']:.4f}** | {snippet} |")


--- Relevance Scores (Summary vs. Articles) ---
| Rank | Relevance Score | Article Snippet |
|------|-------------------|-----------------|
| 1    | **0.5799** | The latest article on CPU architecture details how Intel's n... |
| 2    | **0.4934** | Quantum computing breakthroughs lead to the first stable 50-... |
